In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys    
directory = '/content/drive/My Drive/Research Papers/IGPL 2022/Code'
sys.path.append(directory)

In [3]:
from suma_fx import ari

In [4]:
ari.suma(2,2)

4